In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
 
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Q1").getOrCreate()

In [ ]:
from pyspark import SparkFiles
from google.colab import files

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
print(df.count())

4850360


In [ ]:
df = df.dropna()
df = df.dropDuplicates()
print(df.count())

4849000


In [ ]:
review_id__df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id__df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100E6OT70LKOZ|   21498849|B001E29RG6|     221089601| 2011-02-27|
|R100JZDN9RV012|   28299024|B0035HEPXS|     173761747| 2011-10-05|
|R100WJRDBA7IQ5|   40179799|B0019GJZXW|     704428554| 2014-06-18|
|R100ZY64ZJVNVZ|   25717098|B000MMZ5ZG|     247901753| 2015-02-23|
|R101GPEDWLA42J|   42195961|B00U821IME|     647954521| 2015-05-09|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
products_df = df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001E29RG6|Dallas Cowboys Lu...|
|B0035HEPXS|Smooth Fitness 5....|
|B0019GJZXW|SKLZ Baseball Spe...|
|B000MMZ5ZG|Cannon Low-Profil...|
|B00U821IME|5 Pcs Assort Agat...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
customers_df = df.groupby('customer_id').sum()
customers_df.show(5)

+-----------+----------------+-------------------+------------------+----------------+
|customer_id|sum(customer_id)|sum(product_parent)|sum(helpful_votes)|sum(total_votes)|
+-----------+----------------+-------------------+------------------+----------------+
|    1513285|        10592995|         3442197836|                 0|               1|
|    1924038|        13468266|         3318351791|                 2|               4|
|    5625756|        16877268|         1883027268|                 0|               0|
|   48333068|       338331476|         3207813052|                 5|               5|
|   37443191|       149772764|         1914716306|                 3|               5|
+-----------+----------------+-------------------+------------------+----------------+
only showing top 5 rows



In [ ]:
customers_df = customers_df.select(['customer_id', 'sum(customer_id)'])
customers_df.withColumnRenamed('sum(customer_id)', 'customer_count')
customers_df.show(5)

+-----------+----------------+
|customer_id|sum(customer_id)|
+-----------+----------------+
|    1513285|        10592995|
|    1924038|        13468266|
|    5625756|        16877268|
|   48333068|       338331476|
|   37443191|       149772764|
+-----------+----------------+
only showing top 5 rows



In [ ]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100E6OT70LKOZ|          3|            0|          0|   N|
|R100JZDN9RV012|          5|            1|          2|   N|
|R100WJRDBA7IQ5|          5|            0|          0|   N|
|R100ZY64ZJVNVZ|          5|            0|          0|   N|
|R101GPEDWLA42J|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('config.py','wb').write(src)
from config import password, rds_url

Saving config.py to config (1).py


In [ ]:
mode="append"
jdbc_url = f"jdbc:postgresql://{rds_url}:5432/amazon_db"
config = {"user":"root",
          "password": f"{password}",
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id__df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)